In [ ]:
!pip install torch-geometric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#Import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


AttributeError: module 'torch' has no attribute 'version'

In [2]:
!nvcc --version

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [147]:
#Test for new dataset
def testFinal(model, indata, device):
    model.eval()
    title = ''
    E_A = 0
    E_A_Ad = 0
    E_A_Ad2 = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          epsilon = 1e-8
          for i in range(len(out)):
            e = ((out[i].item() - real[i].item() ) / (epsilon + out[i].item()))*100
            errorList.append(e)

          errorListAbs = [abs(x) for x in errorList]

          errorListAbsAd2 = errorListAbs[8:12]

          errorListAbs.sort()
          errorListAbsAd = errorListAbs[:-2]

          E_A_el = sum(errorListAbs)/len(errorListAbs)
          E_A_Ad_el = sum(errorListAbsAd)/len(errorListAbsAd)
          E_A_Ad_el2 = sum(errorListAbsAd2)/len(errorListAbsAd2)

          E_A += E_A_el
          E_A_Ad += E_A_Ad_el
          E_A_Ad2 += E_A_Ad_el2
          
          print([round(x,2) for x in errorList])
          print('Avarage Error: '+ str(round(E_A_el,2))+'%, Adjusted Avarage Error: '+ str(round(E_A_Ad_el,2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),4))+'\n')

          #errorAvg += sum(errorListAbs)/len(errorListAbs)

    print('Avarage Tot. Error: '+ str(round(E_A/len(indata),2))+'%\n')
    print('After removing the two worst errors. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad/len(indata),2))+'%')
    print('Total error for nodes 8, 9, 10, 11. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad2/len(indata),2))+'%')
   
#Test for new dataline

def testData(tester_model, datasetTest,line):
    real = datasetTest[line].y
    
    out = tester_model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )*100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

In [103]:
#Traindata

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    #zipped = list(zip(dataX, dataY))
    #np.random.shuffle(zipped)
    #dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset

#Getting data
datasetTrain = get_data("1DatasetTrain.txt")
datasetTest = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\1data\\1DatasetTest.txt")


In [164]:
#Data setup
datasetNew = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\1data\\1DatasetNew.txt")
print(len(datasetNew))
print(len(datasetTrain))
print(len(datasetTest))

#Splitting dataset

#train_loader = datasetTrain[:int(len(datasetTrain)*0.75)]
#test_loader = datasetTrain[int(len(datasetTrain)*0.75):]

train_loader = datasetTrain[:4000]
test_loader = datasetTrain[-1000:]

print(len(train_loader))
print(len(test_loader))



100
6571
1001
4000
1000


In [91]:
#Data Checks
""""
# one data object from train_loader:
graph = train_loader[55]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)
print(graph.x, graph.y)
"""

3750
1250
1475
14746


'"\n# one data object from train_loader:\ngraph = train_loader[55]\nprint(graph.x.shape)\nprint(graph.edge_index.shape)\nprint(graph.y.shape)\nprint(graph.x, graph.y)\n'

In [79]:
# ArchNN1

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x


In [98]:
# ArchNN2

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x



In [121]:

import time
import torch
import torch.nn.functional as F

def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)

def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)

def lossAtan(output, real):
    epsilon = 1e-7
    error = ((real - output) / ((real) + epsilon)) 
    error = torch.clamp(error, max=1)   
    return (torch.mean(torch.atan(error) * 100))



def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))

        #loss = F.mse_loss(output, data.y.to(device))
        loss = F.l1_loss(output, data.y.to(device))
        #loss = lossAtan(output, data.y.to(device))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            total_loss += F.l1_loss(out, data.y.to(device))

            #F.mse_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)


In [165]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=8).to(device_name)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

run(model, train_loader, test_loader, num_nodes, 300, optimizer, device_name)
torch.save(model.state_dict(), "2304-data1-3000-e300_state_dict.pt")

Epoch: 1/300, Duration: 95.272s,Train Loss: 0.7896, Test Loss:0.6038
Epoch: 2/300, Duration: 161.307s,Train Loss: 0.5292, Test Loss:0.5012
Epoch: 3/300, Duration: 214.674s,Train Loss: 0.4846, Test Loss:0.4849
Epoch: 4/300, Duration: 221.990s,Train Loss: 0.4609, Test Loss:0.4286
Epoch: 5/300, Duration: 133.253s,Train Loss: 0.4078, Test Loss:0.2914
Epoch: 6/300, Duration: 135.043s,Train Loss: 0.2534, Test Loss:0.2029
Epoch: 7/300, Duration: 135.155s,Train Loss: 0.1801, Test Loss:0.1258
Epoch: 8/300, Duration: 135.657s,Train Loss: 0.1394, Test Loss:0.1190
Epoch: 9/300, Duration: 135.254s,Train Loss: 0.1136, Test Loss:0.1075
Epoch: 10/300, Duration: 135.493s,Train Loss: 0.1066, Test Loss:0.0748
Epoch: 11/300, Duration: 136.181s,Train Loss: 0.1039, Test Loss:0.0634
Epoch: 12/300, Duration: 137.195s,Train Loss: 0.0974, Test Loss:0.0691
Epoch: 13/300, Duration: 138.066s,Train Loss: 0.0959, Test Loss:0.0675
Epoch: 14/300, Duration: 139.626s,Train Loss: 0.0864, Test Loss:0.0844
Epoch: 15/300, D

In [152]:
torch.save(model.state_dict(), "2304-data1-3000-e300_state_dict.pt")

#loading
tester_model = ArchNN(num_features, num_nodes, heads=8) 
tester_model.load_state_dict(torch.load("2204-data1-e300_state_dict.pt")) 

tester_model.eval()

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [150]:
#More epochs per favoure?
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]
run(model, train_loader, test_loader, num_nodes, 200, optimizer, device_name)

Epoch: 1/200, Duration: 25.122s,Train Loss: 0.0785, Test Loss:0.0688
Epoch: 2/200, Duration: 25.369s,Train Loss: 0.0733, Test Loss:0.0747
Epoch: 3/200, Duration: 23.211s,Train Loss: 0.0731, Test Loss:0.0904
Epoch: 4/200, Duration: 21.029s,Train Loss: 0.0776, Test Loss:0.1056
Epoch: 5/200, Duration: 21.270s,Train Loss: 0.0744, Test Loss:0.0772
Epoch: 6/200, Duration: 21.445s,Train Loss: 0.0707, Test Loss:0.0736
Epoch: 7/200, Duration: 21.323s,Train Loss: 0.0773, Test Loss:0.0768
Epoch: 8/200, Duration: 21.341s,Train Loss: 0.0790, Test Loss:0.0869
Epoch: 9/200, Duration: 21.104s,Train Loss: 0.0819, Test Loss:0.0616
Epoch: 10/200, Duration: 21.616s,Train Loss: 0.0742, Test Loss:0.0650
Epoch: 11/200, Duration: 22.394s,Train Loss: 0.0691, Test Loss:0.0515
Epoch: 12/200, Duration: 21.241s,Train Loss: 0.0756, Test Loss:0.0812
Epoch: 13/200, Duration: 21.295s,Train Loss: 0.0792, Test Loss:0.0752
Epoch: 14/200, Duration: 21.283s,Train Loss: 0.0743, Test Loss:0.0743
Epoch: 15/200, Duration: 20.8

In [166]:
#Test for new dataset

testFinal(tester_model, datasetTest, device_name)


[-1.67, 3.81, 0.92, 0.21, -45.03, 3.41, -0.19, -1.34, -1.7, -0.64, 1.47, 1.48, 1.39, 4.41, 8.51, -75.77, -0.7, -1.16, -1.77, -2.15]
Avarage Error: 7.89%, Adjusted Avarage Error: 2.05%
Mean Square: 0.0033

[-0.73, 3.93, 0.26, 1.51, 1.48, 1.69, 2.9, -1.58, -1.38, -1.98, -0.84, -0.54, -3.47, 5.52, 0.98, 1.72, 2.18, 1.76, -2.69, -1.91]
Avarage Error: 1.95%, Adjusted Avarage Error: 1.65%
Mean Square: 0.001

[6.27, 2.62, 3.24, 7.74, 3.26, -9.69, 0.06, 0.37, -0.09, 0.58, 3.34, 5.04, 4.18, -0.82, 1.04, 2.02, 2.6, 1.97, 1.57, 3.04]
Avarage Error: 2.98%, Adjusted Avarage Error: 2.34%
Mean Square: 0.0007

[-2.47, 0.67, -0.27, 3.59, 2.39, 18.72, -10.96, -1.5, -1.59, -1.92, 0.25, 0.6, -1.03, 1.17, -19.72, -0.78, 1.3, 3.14, 5.02, 23.24]
Avarage Error: 5.02%, Adjusted Avarage Error: 3.19%
Mean Square: 0.0004

[4.19, 1.81, 2.54, 4.88, 17.36, -21.25, -7.99, -1.35, 0.33, 0.63, 0.77, 1.47, 2.61, -3.51, -2.26, 0.52, 0.48, 2.79, 1.79, 1.89]
Avarage Error: 4.02%, Adjusted Avarage Error: 2.32%
Mean Square: 0

In [158]:
#Test for new dataline

testData(model, datasetTest,1)

real      out       %error    Sign      
4.303     4.272     -0.73     True      
-0.844    -0.879    3.93      True      
1.293     1.297     0.26      True      
2.235     2.270     1.51      True      
2.203     2.237     1.48      True      
1.485     1.510     1.69      True      
0.398     0.409     2.90      True      
-0.747    -0.735    -1.58     True      
-1.677    -1.654    -1.38     True      
-2.195    -2.152    -1.98     True      
-2.195    -2.177    -0.84     True      
-1.677    -1.668    -0.54     True      
-0.747    -0.722    -3.47     True      
0.398     0.421     5.52      True      
1.485     1.500     0.98      True      
2.203     2.242     1.72      True      
2.235     2.285     2.18      True      
1.293     1.316     1.76      True      
-0.844    -0.822    -2.69     True      
-4.303    -4.222    -1.91     True      


In [155]:
print(tester_model)

ArchNN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [153]:
testFinal(model, datasetNew, device_name)

[1.72, 0.83, -2.76, 1.0, 0.85, 1.32, -0.06, -1.37, -1.51, -2.7, -2.56, -2.56, -3.53, 0.7, -0.68, -0.06, -0.53, 1.05, -8.9, -2.6]
Avarage Error: 1.86%, Adjusted Avarage Error: 1.38%
Mean Square: 0.0028

[1.77, 0.68, -2.91, 0.94, 0.83, 1.32, -0.03, -1.42, -1.58, -2.75, -2.6, -2.61, -3.46, 0.81, -0.66, -0.03, -0.56, 1.01, -9.02, -2.64]
Avarage Error: 1.88%, Adjusted Avarage Error: 1.4%
Mean Square: 0.003

[1.82, 0.44, -3.03, 0.89, 0.8, 1.3, -0.02, -1.47, -1.64, -2.81, -2.66, -2.65, -3.38, 0.94, -0.63, 0.01, -0.6, 0.97, -9.14, -2.7]
Avarage Error: 1.89%, Adjusted Avarage Error: 1.41%
Mean Square: 0.0032

[1.84, 0.12, -3.18, 0.83, 0.77, 1.28, -0.03, -1.51, -1.71, -2.87, -2.72, -2.7, -3.3, 1.06, -0.6, 0.04, -0.65, 0.96, -9.23, -2.75]
Avarage Error: 1.91%, Adjusted Avarage Error: 1.42%
Mean Square: 0.0034

[1.86, -0.24, -3.35, 0.76, 0.73, 1.26, -0.05, -1.55, -1.79, -2.94, -2.79, -2.77, -3.19, 1.17, -0.59, 0.06, -0.7, 0.96, -9.33, -2.8]
Avarage Error: 1.94%, Adjusted Avarage Error: 1.46%
Mean 